## Importando bibliotecas

In [1]:
import sklearn
import numpy as np
import pandas as pd

## Importando Data

In [2]:
# training data
train = pd.read_csv("./train_pt.csv")

# test data
test = pd.read_csv("./tweets-29-10-2022.csv")

## Explorando o Dataset

In [3]:
train.head(20)

,id,hate,text
0,1,yes,Votaram no PEZAO Agora tomem no CZAO
1,2,no,cuidado com a poupanca pessoal Lembram o que a...
2,3,no,Sabe o que eu acho engracado os nossos governa...
3,4,no,Podiam retirar dos lucros dos bancos
4,5,no,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...
5,6,no,Nao e possivel que deputados tenham tantos car...
6,7,no,Sejam honestos aprovem o projeto original vamo...
7,8,no,Nao a pizza anunciada de anistia do caixa Sim...
8,9,no,quem ja viu bandidos fazendo leis contra antic...
9,10,no,Olha os caras fazendo algo de util e de intere...


In [4]:
test.tail()

,trend,id,text
94,Gabigol,1586531300848337000,RT @camilavicentea: “Agora o que nos resta? An...
95,Gabigol,1586531298474356700,RT @brunoformiga: Existe a Era Zico. E existe ...
96,Gabigol,1586531304929169400,"RT @gabigol: Deus Obrigado por tudo, pela sua ..."
97,Gabigol,1586531298025226200,RT @gabigol: BICAMPEÃO! 👑🏆 #M3NGO @Flamengo...
98,Gabigol,1586531297115000800,"@gabigol Gabriel, obrigado por tudo"


In [5]:
label_map = {
    'yes': 1,
    'no': 0
}

train['label'] = train.hate.map(label_map)

In [6]:
train.head()

,id,hate,text,label
0,1,yes,Votaram no PEZAO Agora tomem no CZAO,1
1,2,no,cuidado com a poupanca pessoal Lembram o que a...,0
2,3,no,Sabe o que eu acho engracado os nossos governa...,0
3,4,no,Podiam retirar dos lucros dos bancos,0
4,5,no,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...,0


In [7]:
# Count non hate speech related tweets
sum(train["label"] == 0)

831

In [28]:
train["label"]

0       1
1       0
2       0
3       0
4       0
       ..
1028    0
1029    0
1030    1
1031    0
1032    0
Name: label, Length: 1033, dtype: int64

In [8]:
# Count hate speech related tweets
sum(train["label"] == 1)

202

In [9]:
train.shape

(1033, 4)

In [10]:
# check if there are any missing values
train.isnull().sum()
#train.isnull().values.any()

id       0
hate     0
text     0
label    0
dtype: int64

## Limpeza dos Dados

In [11]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [12]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [13]:
import preprocessor as p

# Função para limpar o Dataset (utiliza o tweet_preprocessor e expressões regulares)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # converte todos os tweets para lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [14]:
# clean training data
train_clean_tweet = clean_tweets(train["text"])
train_clean_tweet = pd.DataFrame(train_clean_tweet)

In [15]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_clean_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,id,hate,text,label,clean_tweet
0,1,yes,Votaram no PEZAO Agora tomem no CZAO,1,votaram no pezao agora tomem no czao
1,2,no,cuidado com a poupanca pessoal Lembram o que a...,0,cuidado com a poupanca pessoal lembram o que a...
2,3,no,Sabe o que eu acho engracado os nossos governa...,0,sabe o que eu acho engracado os nossos governa...
3,4,no,Podiam retirar dos lucros dos bancos,0,podiam retirar dos lucros dos bancos
4,5,no,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...,0,cade o galvao pra narrar agora faliiiiuuuuuuu ...
5,6,no,Nao e possivel que deputados tenham tantos car...,0,nao e possivel que deputados tenham tantos car...
6,7,no,Sejam honestos aprovem o projeto original vamo...,0,sejam honestos aprovem o projeto original vamo...
7,8,no,Nao a pizza anunciada de anistia do caixa Sim...,0,nao a pizza anunciada de anistia do caixa sim ...
8,9,no,quem ja viu bandidos fazendo leis contra antic...,0,quem ja viu bandidos fazendo leis contra antic...
9,10,no,Olha os caras fazendo algo de util e de intere...,0,olha os caras fazendo algo de util e de intere...


## Separando dados para teste e Treino

In [16]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

## Vetorizando tweets com CountVectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
documents = ["This is our TGI tesis test algorithm",
             "Data science is my passion and it is fun!",
             "This is an awesome short paper"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names_out())

,algorithm,an,and,awesome,data,fun,is,it,my,our,paper,passion,science,short,tesis,test,tgi,this
0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,1,1
1,0,0,1,0,1,1,2,1,1,0,0,1,1,0,0,0,0,0
2,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1


In [68]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True)

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

## Contruindo o Modelo

Aplicando Support Vector Classifier (SVC)

In [69]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

## Verificando a precisão do SVC

In [70]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')



Accuracy score for SVC is:  79.35483870967742 %


# Testes

In [18]:
test_clean_tweet = clean_tweets(test["text"])
test_clean_tweet = pd.DataFrame(test_clean_tweet)

In [19]:
# append cleaned tweets to the training data
test["clean_tweet"] = test_clean_tweet

# compare the cleaned and uncleaned tweets
test.head(10)

,trend,id,text,clean_tweet
0,#LulaPresidente1️⃣3️⃣,1586531315318689800,"Em nome de jesus ,amém🙏🙌 #LulaPresidente1️⃣3️⃣...",em nome de jesus amm
1,#LulaPresidente1️⃣3️⃣,1586531316727566300,"RT @MrJimmyDaFloof: #NewProfilePic amanhã, car...",amanh caralho
2,#LulaPresidente1️⃣3️⃣,1586531314022240300,#LulaPresidente1️⃣3️⃣ pq o Brasil merece um ...,pq o brasil merece um presidente melhor
3,#LulaPresidente1️⃣3️⃣,1586531305252003800,RT @jandira_feghali: Numa escala de 10 a 10 qu...,numa escala de a qual o nvel de esperana de vocs
4,#LulaPresidente1️⃣3️⃣,1586531309765087200,RT @ptsaopaulosp: A VIRADA CHEGOU!!! Vamos que...,a virada chegou vamos que vamos que amanh
5,#LulaPresidente1️⃣3️⃣,1586531306640638000,RT @Fagner_jp: Tem cidadão de bem que apoia es...,tem cidado de bem que apoia esse tipo de gent...
6,#LulaPresidente1️⃣3️⃣,1586531302789894100,@guinasouzacanal @AndreJanonesAdv É sobre isso...,sobre isso
7,#LulaPresidente1️⃣3️⃣,1586531305176518700,"@folha Só por isso ai, de armação fake-fakeada...",s por isso ai de armao fake fakeada2018 e para...
8,TE AMO ESPANHOLA,1586531313095352300,@direitasiqueira BOLSONARISMO MATA ZAMBELLI NA...,bolsonarismo mata zambelli na cadeia te amo es...
9,#LulaPresidente1️⃣3️⃣,1586531300227174400,Essa chuva que tá caindo já é lavando o bolson...,essa chuva que t caindo j lavando o bolsonaris...


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True)

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test) + list(test.clean_tweet.values))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)
test_vec = vectorizer.transform(test.clean_tweet.values)

In [23]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(test_vec)

In [27]:
y_pred_svm

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [25]:
test.shape

(99, 4)

In [30]:
test["label"] = pd.DataFrame(y_pred_svm)

In [36]:
test.head(20)

,trend,id,text,clean_tweet,label
0,#LulaPresidente1️⃣3️⃣,1586531315318689800,"Em nome de jesus ,amém🙏🙌 #LulaPresidente1️⃣3️⃣...",em nome de jesus amm,0
1,#LulaPresidente1️⃣3️⃣,1586531316727566300,"RT @MrJimmyDaFloof: #NewProfilePic amanhã, car...",amanh caralho,0
2,#LulaPresidente1️⃣3️⃣,1586531314022240300,#LulaPresidente1️⃣3️⃣ pq o Brasil merece um ...,pq o brasil merece um presidente melhor,0
3,#LulaPresidente1️⃣3️⃣,1586531305252003800,RT @jandira_feghali: Numa escala de 10 a 10 qu...,numa escala de a qual o nvel de esperana de vocs,0
4,#LulaPresidente1️⃣3️⃣,1586531309765087200,RT @ptsaopaulosp: A VIRADA CHEGOU!!! Vamos que...,a virada chegou vamos que vamos que amanh,0
5,#LulaPresidente1️⃣3️⃣,1586531306640638000,RT @Fagner_jp: Tem cidadão de bem que apoia es...,tem cidado de bem que apoia esse tipo de gent...,1
6,#LulaPresidente1️⃣3️⃣,1586531302789894100,@guinasouzacanal @AndreJanonesAdv É sobre isso...,sobre isso,0
7,#LulaPresidente1️⃣3️⃣,1586531305176518700,"@folha Só por isso ai, de armação fake-fakeada...",s por isso ai de armao fake fakeada2018 e para...,0
8,TE AMO ESPANHOLA,1586531313095352300,@direitasiqueira BOLSONARISMO MATA ZAMBELLI NA...,bolsonarismo mata zambelli na cadeia te amo es...,0
9,#LulaPresidente1️⃣3️⃣,1586531300227174400,Essa chuva que tá caindo já é lavando o bolson...,essa chuva que t caindo j lavando o bolsonaris...,0


In [32]:
sum(test["label"] == 1)

8

In [42]:
test.loc[test['label'] == 1].clean_tweet

5      tem cidado de bem que apoia esse tipo de gent...
13     eu j assisti o vdeo atentamente por vezes o m...
32     video da carla zambelli correndo atrs do band...
60     se o kiko late eu escuto se o kiko mostra a b...
61                               kkkkkkkk vazou o filme
63                                            alexandre
71     gabriel barbosa eu s quero a tua roula encaix...
82     eu j sou apaixonada por voc guaxi no precisav...
Name: clean_tweet, dtype: object